In [9]:
import pandas as pd

noc_regions = pd.read_csv("noc_regions.csv")

In [10]:
# ci-dessous : on importe le dataframe du monde (en vue d'afficher la carte)

import geopandas as gpd
import matplotlib.pyplot as plt

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

In [11]:
# dictionnaire pour faire matcher "region" de noc_regions avec "SOVEREIGNT" de world 

overrides = { 
    "USA" : "United States of America",
    "Trinidad" : "Trinidad and Tobago",
    "Serbia": "Republic of Serbia",
    "Tanzania" : "United Republic of Tanzania", 
    "UK" : "United Kingdom",
    "Bahamas" : "The Bahamas", 
    "Macedonia" : "North Macedonia",
    "Czech Republic" : "Czechia" 
}

In [12]:
# je souhaite créer un dataframe avec 4 colonnes :
# region et NOC de noc_regions
# SOV_A3 et SOVEREIGNT de world


import pandas as pd


df = (pd.DataFrame
    .from_dict(overrides,
    orient = 'index',
    columns = ["SOVEREIGNT"])
.reset_index()
.rename(columns = {"index" : "short_names"})
)

df = df.merge(
    world[["SOVEREIGNT", "SOV_A3"]],
    on = "SOVEREIGNT", 
    how = 'left'
)

df = df.merge(
    noc_regions[['NOC','region']],
    left_on = "short_names",
    right_on = "region",
    how = 'left'
)

result = df[['region','NOC', 'SOV_A3', 'SOVEREIGNT']]
result

,region,NOC,SOV_A3,SOVEREIGNT
0,USA,USA,US1,United States of America
1,USA,USA,US1,United States of America
2,Trinidad,TTO,TTO,Trinidad and Tobago
3,Trinidad,WIF,TTO,Trinidad and Tobago
4,Serbia,SCG,SRB,Republic of Serbia
5,Serbia,SRB,SRB,Republic of Serbia
6,Serbia,YUG,SRB,Republic of Serbia
7,Tanzania,TAN,TZA,United Republic of Tanzania
8,UK,GBR,GB1,United Kingdom
9,UK,GBR,GB1,United Kingdom


In [13]:
print(result['region'].unique)

<bound method Series.unique of 0                USA
1                USA
2           Trinidad
3           Trinidad
4             Serbia
5             Serbia
6             Serbia
7           Tanzania
8                 UK
9                 UK
10           Bahamas
11         Macedonia
12    Czech Republic
13    Czech Republic
14    Czech Republic
Name: region, dtype: object>


In [14]:
# Maintenant l'objectif est de créer un dictionnaire
# correspondance SOV_A3 -> NOCs
# à partir du dataframe précédent


# on sélectionne les colonnes SOV_A3 et NOC, onlève les N/A
df_full = df[['SOV_A3', 'NOC']].dropna(subset=['SOV_A3','NOC'])

mapping_sov3_to_nocs = (
    df_full
    .groupby('SOV_A3')['NOC']
    .apply(lambda ser: sorted(set(ser)))
    .to_dict()
    )

# print pour vérifier
for sov_a3, nocs in mapping_sov3_to_nocs.items():
    print(f"{sov_a3!r} -> {nocs}")


'BHS' -> ['BAH']
'CZE' -> ['BOH', 'CZE', 'TCH']
'GB1' -> ['GBR']
'MKD' -> ['MKD']
'SRB' -> ['SCG', 'SRB', 'YUG']
'TTO' -> ['TTO', 'WIF']
'TZA' -> ['TAN']
'US1' -> ['USA']


In [17]:
mapping_sov3_to_nocs

{'BHS': ['BAH'],
 'CZE': ['BOH', 'CZE', 'TCH'],
 'GB1': ['GBR'],
 'MKD': ['MKD'],
 'SRB': ['SCG', 'SRB', 'YUG'],
 'TTO': ['TTO', 'WIF'],
 'TZA': ['TAN'],
 'US1': ['USA']}

In [ ]:
mapping_nocs_to_sov3 = {}
for sov3, nocs in mapping_sov3_to_nocs.items():
    for noc in nocs:
        mapping_nocs_to_sov3[noc] = sov3

mapping_nocs_to_sov3

{'BAH': 'BHS',
 'BOH': 'CZE',
 'CZE': 'CZE',
 'TCH': 'CZE',
 'GBR': 'GB1',
 'MKD': 'MKD',
 'SCG': 'SRB',
 'SRB': 'SRB',
 'YUG': 'SRB',
 'TTO': 'TTO',
 'WIF': 'TTO',
 'TAN': 'TZA',
 'USA': 'US1'}

In [ ]:
fix_names = {}

In [18]:
region_to_sovereign = result.set_index('region')['SOVEREIGNT'].to_dict()

In [19]:
region_to_sovereign

{'USA': 'United States of America',
 'Trinidad': 'Trinidad and Tobago',
 'Serbia': 'Republic of Serbia',
 'Tanzania': 'United Republic of Tanzania',
 'UK': 'United Kingdom',
 'Bahamas': 'The Bahamas',
 'Macedonia': 'North Macedonia',
 'Czech Republic': 'Czechia'}